### Solace Relativity v1.0 - Build Relativity and EJ Weighting

Read exploits db, use derivedValueIndicators and build data-values files and relativity scores, build expert-judgement template files, and build expert-judgement weighting files.

In [1]:
import pandas as pd
import json
import numpy as np
import math
import os

NUMBEROFEJSUBMITTERS = 3

#WIP

## Load Exploits, Protocols(tbd), and mappingTables (transactional and temporal) 
# TODO - next version, find each predictor columns and create mapping table entries
exploitsDf = pd.read_csv('../exploits/historical_exploits.csv')
f = open('../governance/reference/mappingTablesSolace.json')
lookupTables = json.loads(f.read())
f.close()

# Init dfs for lookup tables
tvlSolaceLookupTable = pd.DataFrame(lookupTables['tvlSolaceLookup'])
auditsSolaceLookupTable = pd.DataFrame(lookupTables['auditSolaceLookup'])
oraclesSolaceLookupTable = lookupTables['oracleSolaceLookup'] # just values, not mappings: yes/no/unknown
# TODO - category mapping ??

# Calc Exploit Damage ratio
exploitsDf['tvlPreExploit'] = exploitsDf['tvlPreExploit'] / 10 # confirm wtih Tim
exploitsDf['exploitDamage'] = exploitsDf['loss'] / exploitsDf['tvlPreExploit']
#exploitsDf['exploitDamage']= exploitsDf['exploitDamage'].astype(object).replace(np.nan, 'null')

# Calc time to exploit
exploitsDf[['launchDate','exploitDate']] = exploitsDf[['launchDate','exploitDate']].apply(pd.to_datetime) #if conversion required
exploitsDf['timeToExploit'] = (exploitsDf['exploitDate'] - exploitsDf['launchDate']).dt.days
exploitsDf['timeToExploit'] = exploitsDf['timeToExploit'].astype(object).replace(np.nan, 'null')

# Derive tvlSolace column
def lookup_tvlSolace(tvl):
    if math.isnan(tvl):
        return 'null'
    match = (tvlSolaceLookupTable['lowerLimit'] <= tvl) & (tvlSolaceLookupTable['upperLimit'] > tvl)
    tvlOut = tvlSolaceLookupTable['tvlSolace'][match]
    return tvlOut.values[0]
exploitsDf['tvlSolace'] = exploitsDf['tvlPreExploit'].apply(lookup_tvlSolace)

# Derive auditSolace column
def lookup_auditSolace(audits):
    if math.isnan(audits):
        return 'Unknown'
    match = (auditsSolaceLookupTable['auditsOg'] == audits)
    auditsOut = auditsSolaceLookupTable['auditSolace'][match]
    return auditsOut.values[0]
exploitsDf['auditSolace'] = exploitsDf['audits'].apply(lookup_auditSolace)

# Derive oracleSolace column
def lookup_oracleSolace(oracles,catalog):
    if pd.isna(catalog):
        return 'Unknown'
    elif pd.isna(oracles):
        return 'No'
    else:
        return "Yes"
exploitsDf['oracleSolace'] = exploitsDf.apply(lambda x: lookup_oracleSolace(x.oracles, x.category), axis=1)

# Create columns for use in one way calcs
exploitsDf['tvlAvgDamage'] = exploitsDf.groupby(['tvlSolace'])['exploitDamage'].transform('mean')
exploitsDf['auditAvgDamage'] = exploitsDf.groupby(['auditSolace'])['exploitDamage'].transform('mean')
exploitsDf['oracleAvgDamage'] = exploitsDf.groupby(['oracleSolace'])['exploitDamage'].transform('mean')

averageDamage = exploitsDf['exploitDamage'].mean()

exploitsDf['tvlDamageRatio'] = exploitsDf['tvlAvgDamage'] / averageDamage
exploitsDf['auditDamageRatio'] = exploitsDf['auditAvgDamage'] / averageDamage
exploitsDf['oracleDamageRatio'] = exploitsDf['oracleAvgDamage'] / averageDamage

print('Average exploit damage ratio: ',averageDamage)

# Create one way 'tables', separate for readability 
# TVL
tvlOneWay = pd.DataFrame()
tvlOneWay['damageRatio'] = exploitsDf['tvlDamageRatio'].unique()
tvlOneWay['tvlSolace'] =exploitsDf['tvlSolace'].unique()
tvlOneWay['avgDamage'] = exploitsDf['tvlAvgDamage'].unique()

# AUDITS
auditOneWay = pd.DataFrame()
auditOneWay['damageRatio'] = exploitsDf['auditDamageRatio'].unique()
auditOneWay['auditSolace'] = exploitsDf['auditSolace'].unique()
auditOneWay['avgDamage'] = exploitsDf['auditAvgDamage'].unique()

# ORACLES
oracleOneWay = pd.DataFrame()
oracleOneWay['damageRatio'] = exploitsDf['oracleDamageRatio'].unique()
oracleOneWay['oracleSolace'] =exploitsDf['oracleSolace'].unique()
oracleOneWay['avgDamage'] = exploitsDf['oracleAvgDamage'].unique()

def write_json(df, fileName, drop):
    if drop == True:
        df.drop(['avgDamage'], axis=1, inplace=True)
    data = df.to_json(orient='columns')
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    with open(fileName, 'w') as outfile:
        outfile.write(data)

write_json(pd.merge(tvlOneWay, tvlSolaceLookupTable, how="right", on='tvlSolace'), '../governance/data-values/tvlDataValues.json', True)
write_json(pd.merge(auditOneWay, auditsSolaceLookupTable, how="right", on='auditSolace'), '../governance/data-values/auditDataValues.json', True)
write_json(oracleOneWay, '../governance/data-values/oracleDataValues.json', True)

print("Data values json files created in governance directory")

# Initialize EJ directories, files and weightings
def initialize_ejs(df):
    for i in df['damageRatio']:
        df['damageRatio'] = 1
    return df

# EJ weighting files
def create_weights(predictor):
    weightingsTemplate = '{"' + predictor + 'Basis": {"1": "data"},"weight": {"1": 1}}'
    weightingsTemplateJson = json.loads(weightingsTemplate)
    weightingsTemplateDf = pd.DataFrame(weightingsTemplateJson)
    write_json(weightingsTemplateDf, '../governance/weightings/'+predictor+'Weighting.json', False)
create_weights('audit')
create_weights("oracle")
create_weights("tvl")

for i in range(NUMBEROFEJSUBMITTERS):
    print("Initializing EJ files " + str(i+1) + "...")
    write_json(initialize_ejs(tvlOneWay), '../governance/expert-judgement/' + str(i+1) + '/ejTvlDataValues.json', False)
    write_json(initialize_ejs(oracleOneWay), '../governance/expert-judgement/' + str(i+1) + '/ejOracleDataValues.json', False)
    write_json(initialize_ejs(auditOneWay), '../governance/expert-judgement/' + str(i+1) + '/ejAuditDataValues.json', False)
    print("EJ Submitter" + str(i+1) + " created in governance directory")
    # Create weighting files per predictor with basis and each expert   

print("\nRelativities have been initialed to 1. Update files in governance directory to incorporate expert judgement into realativity scoring.")



Average exploit damage ratio:  0.24146191913870418
Data values json files created in governance directory
Initializing EJ files 1...
EJ Submitter1 created in governance directory
Initializing EJ files 2...
EJ Submitter2 created in governance directory
Initializing EJ files 3...
EJ Submitter3 created in governance directory

Relativities have been initialed to 1. Update files in governance directory to incorporate expert judgement into realativity scoring.
